In [12]:
import pandas as pd
import numpy as np
import pulp as p

### Functions ###

class optimiser:
    def __init__(self, formulations, rm_costs):
        self.formulations = formulations
        self.rm_costs = rm_costs
        
    def formulation_opt (self):

        rms = list(self.formulations.columns)[0:-1] 
        combined_df = self.formulations.copy()

        combined_df = combined_df.rename(columns = {'Batch Size':'Batch Size (kg)'})

        for i in rms:
             combined_df[str(i)] = combined_df[str(i)]*list(self.rm_costs[str(i)]) #can't just be multiplied by df column

        combined_df['Total Cost (£)'] = combined_df[rms].sum(axis = 1)
        combined_df['Cost per kg (£)'] = combined_df['Total Cost (£)']/combined_df['Batch Size (kg)']

        for i in rms:
            combined_df[str(i)] = self.formulations[str(i)]

        self.rm_costs = self.rm_costs.rename(columns = {'Cost per kg':'Cost per kg (£)'})

        nan_costs = list(self.rm_costs.T[np.isnan(self.rm_costs.T['Cost per kg'])].index)
        filtered_df = combined_df.copy()

        for i in nan_costs:
            filtered_df = filtered_df[filtered_df[i] == 0]
        filtered_rms = list(filtered_df.columns)[0:-3]    

        for i in filtered_rms:
            filtered_df[str(i)] = self.formulations[str(i)]

    #     standard_cost_per_batch = filtered_df.iloc[0,-2]
    #     filtered_df ['Cost reduction compared to standard per batch'] = combined_df['Total Cost (£)'] - standard_cost_per_batch

        standard_cost_per_kg = filtered_df.iloc[0,-1]
        filtered_df['Cost difference compared to standard per kg (£)'] = combined_df['Cost per kg (£)'] - standard_cost_per_kg
        filtered_df = filtered_df.sort_values(by = 'Cost per kg (£)', ascending = True)
        
        ### Remove zero columns (optional) ###
    # filtered_df = filtered_df.replace(0,np.NaN).dropna(how = 'all', axis = 1).replace(np.NaN,0)

#         final_frame = pd.DataFrame(data = filtered_df.iloc[0])
#         final_frame = final_frame[(final_frame.T != 0).any()]

#         return final_frame
        return filtered_df


def stp_lp_optimiser(name:str, constants_weights:dict, decision_variables:list, costs:dict, constraints_list:list = None):

#     constants = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
#                  'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VAMMO': 17.9, 'XDINP': 241.8}

#     decision_variables = ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM']

    decision_vars = p.LpVariable.dicts('decision',decision_variables,lowBound=0,cat='Continuous')

    problem = p.LpProblem('dynamic_formulation_problem', p.LpMinimize)
    problem += p.lpSum([costs[i]*decision_vars[i] for i in decision_variables])
    for constraint_dict in constraints_list:
        for key, (sign, var) in constraint_dict.items():
            if sign == '=':
                problem += sum([decision_vars[i] for i in var]) == key
            elif sign == '<=' or sign == '=<':
                problem += sum([decision_vars[i] for i in var]) <= key
            elif sign == '>=' or sign == '=>':
                problem += sum([decision_vars[i] for i in var]) >= key
            
#     problem += decision_vars['VSHI4BTF'] + decision_vars['VPOLSHI17C'] + decision_vars['VSTPE10'] + decision_vars['VSPUR1015LM'] == 356.7
#     problem += decision_vars['VSHI4BTF'] <= 237.8
    #print(problem)
    problem.solve()
    # print(p.value(problem.objective))
    # for v in problem.variables():
    #     print(v.name, "=", v.varValue)

    total_cost1 = p.value(problem.objective) + sum([constants_weights[i]*costs[i] for i in constants_weights]) #total cost of batch
    total_mass1 = sum([v.varValue for v in problem.variables()]) + sum(constants_weights.values()) #total mass of batch
    cost_per_unit1 = total_cost1/total_mass1
    cost_per_unit1

    polymers1 = pd.Series([v.varValue for v in problem.variables()], index = [v.name for v in problem.variables()])
    polymers1 = pd.DataFrame(polymers1, columns=[name])
    constraints1 = pd.DataFrame.from_dict(constants_weights, orient='index', columns=[name])
    lpframe = pd.concat([polymers1, constraints1], axis =0).T
    lpframe['Batch Size (kg)'] = total_mass1
    lpframe['Cost per kg (£)'] = cost_per_unit1
    lpframe = lpframe.rename(columns = {'decision_VPOLSHI17C':'VPOLSHI17C'}).rename(columns = {'decision_VSHI4BTF':'VSHI4BTF'}).rename(columns = {'decision_VSPUR1015LM':'VSPUR1015LM'}).rename(columns = {'decision_VSTPE10':'VSTPE10'})
    return lpframe

In [2]:
### data import ###

costs = {'VOMNYA5ML': 0.15, 'VCRAYVALLACSLT': 7.35, 'VDAMO': 7.435, 'VAMMO': 5.148, 'VDYNASYLAN1146': 16.443, 'VGF80': 8.76,
         'VGF31': 5.217, 'VALINK15': 36.087, 'VHAKUENKACCRS': 0.589, 'VPOLSHI17C': 2.983, 'VSTPE10': 4.217, 'VSHI4BTF': 2.504,
         'VXL10': 2.8, 'XTITAN2': 2.35, 'XDINP': 0.926, 'VDIDP': 1.539, 'VDINCH': 1.84, 'VDESMOPHEN1111': 2.15, 'VCAT850': 21.4,
         'VSPUR1015LM': 4.391}

rms = ['VOMNYA5ML', 'VCRAYVALLACSLT', 'VDAMO', 'VAMMO', 'VHAKUENKACCRS', 'VPOLSHI17C', 'STPE10', 
       'VSHI4BTF', 'VXL10', 'XTITAN2', 'XDINP', 'VDIDP', 'VDINCH', 'VCAT850', 'VSPUR1015LM']

costsdf = pd.DataFrame.from_dict(costs, orient='index').reset_index()#, columns=['RM', ]
costsdf = costsdf.rename(columns = {0:'Cost per kg'}).rename(columns = {'index':'RM'})
costsdf = costsdf.set_index('RM').T

stp_data = pd.read_csv('Point formulations.csv')
stp_data = stp_data.rename(columns = {'Unnamed: 0':'Formulation'})
stp_data = stp_data.set_index('Formulation')
stp_data ['Batch Size'] = stp_data.sum(axis = 1)
costsdf

RM,VOMNYA5ML,VCRAYVALLACSLT,VDAMO,VAMMO,VDYNASYLAN1146,VGF80,VGF31,VALINK15,VHAKUENKACCRS,VPOLSHI17C,VSTPE10,VSHI4BTF,VXL10,XTITAN2,XDINP,VDIDP,VDINCH,VDESMOPHEN1111,VCAT850,VSPUR1015LM
Cost per kg,0.15,7.35,7.435,5.148,16.443,8.76,5.217,36.087,0.589,2.983,4.217,2.504,2.8,2.35,0.926,1.539,1.84,2.15,21.4,4.391


In [3]:
point_formulations = optimiser(stp_data, costsdf).formulation_opt()
point_formulations = point_formulations.drop(['Cost difference compared to standard per kg (£)', 'Total Cost (£)'], axis = 1)
point_formulations = point_formulations.fillna(0)
point_formulations

,VSHI4BTF,VSPUR1015LM,VSTPE10,VPOLSHI17C,XDINP,VDINCH,VDAMO,VAMMO,VCRAYVALLACSLT,VHAKUENKACCRS,VXL10,VOMNYA5ML,VCAT850,XTITAN2,VDIDP,Batch Size (kg),Cost per kg (£)
Formulation,,,,,,,,,,,,,,,,,
Standard,237.8,0.0,0.0,118.9,241.8,0.0,14.9,0.0,19.50,780,65,260,3.3,13,29.3,1783.50,1.264270
1,0.0,0.0,356.7,0.0,271.1,0.0,0.0,17.9,9.75,780,65,260,0.0,13,0.0,1773.45,1.482999
3,0.0,0.0,356.7,0.0,271.1,0.0,14.9,0.0,19.50,780,65,260,0.0,13,0.0,1780.20,1.528097
4,0.0,0.0,356.7,0.0,0.0,271.1,14.9,0.0,11.70,780,65,260,0.0,13,0.0,1772.40,1.642278
2,0.0,0.0,356.7,0.0,0.0,271.1,0.0,17.9,19.50,780,65,260,0.0,13,0.0,1783.20,1.654033


In [16]:
decision_variables = ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM']

### LP1 Polymer for constant VAMMO & VDINP ###
constraints_dict1 = [{356.7: ('=', ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM'])}, {237.8: ('<=', ['VSHI4BTF'])}]

constants1 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
             'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VAMMO': 17.9, 'XDINP': 241.8}

### LP2 Polymer for constant VAMMO & VDINCH ###
constraints_dict2 = [{356.7: ('=', ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM'])}, {237.8: ('<=', ['VSHI4BTF'])}]

constants2 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
             'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VAMMO': 17.9, 'VDINCH': 241.8}

### LP3 Polymer for constant VDAMO & VDINP ###
constraints_dict3 = [{356.7: ('=', ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM'])}, {237.8: ('<=', ['VSHI4BTF'])}]

constants3 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
             'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VDAMO': 14.9, 'XDINP': 241.8}

### LP4 Polymer for constant VDAMO & VDINCH ###
constraints_dict4 = [{356.7: ('=', ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM'])}, {237.8: ('<=', ['VSHI4BTF'])}]

constants4 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
             'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VDAMO': 14.9, 'VDINCH': 241.8}

lp1frame = stp_lp_optimiser('LP1', constants1, decision_variables, costs, constraints_dict1)
lp2frame = stp_lp_optimiser('LP2', constants2, decision_variables, costs, constraints_dict2)
lp3frame = stp_lp_optimiser('LP3', constants3, decision_variables, costs, constraints_dict3)
lp4frame = stp_lp_optimiser('LP4', constants4, decision_variables, costs, constraints_dict4)

In [108]:
# # ### LP1 Polymer for constant VAMMO & VDINP ###

# # constants1 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
# #              'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VAMMO': 17.9, 'XDINP': 241.8}

# # decision_variables = ['VSHI4BTF', 'VPOLSHI17C', 'VSTPE10', 'VSPUR1015LM']

# # decision_vars = p.LpVariable.dicts('decision',decision_variables,lowBound=0,cat='Continuous')

# # problem1 = p.LpProblem('dynamic_formulation_problem', p.LpMinimize)
# # problem1 += p.lpSum([costs[i]*decision_vars[i] for i in decision_variables])
# # problem1 += decision_vars['VSHI4BTF'] + decision_vars['VPOLSHI17C'] + decision_vars['VSTPE10'] + decision_vars['VSPUR1015LM'] == 356.7
# # problem1 += decision_vars['VSHI4BTF'] <= 237.8
# # #print(problem)
# # problem1.solve()
# # # print(p.value(problem1.objective))
# # # for v in problem1.variables():
# # #     print(v.name, "=", v.varValue)
    
# # total_cost1 = p.value(problem1.objective) + sum([constants1[i]*costs[i] for i in constants1]) #total cost of batch
# # total_mass1 = sum([v.varValue for v in problem1.variables()]) + sum(constants1.values()) #total mass of batch
# # cost_per_unit1 = total_cost1/total_mass1
# # cost_per_unit1

# # polymers1 = pd.Series([v.varValue for v in problem1.variables()], index = [v.name for v in problem1.variables()])
# # polymers1 = pd.DataFrame(polymers1, columns=['LP1'])
# # constraints1 = pd.DataFrame.from_dict(constants1, orient='index', columns=['LP1'])
# # lp1frame = pd.concat([polymers1, constraints1], axis =0).T
# # lp1frame['Batch Size (kg)'] = total_mass1
# # lp1frame['Cost per kg (£)'] = cost_per_unit1
# # lp1frame = lp1frame.rename(columns = {'decision_VPOLSHI17C':'VPOLSHI17C'}).rename(columns = {'decision_VSHI4BTF':'VSHI4BTF'}).rename(columns = {'decision_VSPUR1015LM':'VSPUR1015LM'}).rename(columns = {'decision_VSTPE10':'VSTPE10'})
# # # lp1frame


# ### LP2 Polymer for constant VAMMO & VDINCH ###

# constants2 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
#              'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VAMMO': 17.9, 'VDINCH': 241.8}

# problem2 = p.LpProblem('dynamic_formulation_problem', p.LpMinimize)
# problem2 += p.lpSum([costs[i]*decision_vars[i] for i in decision_variables])
# problem2 += decision_vars['VSHI4BTF'] + decision_vars['VPOLSHI17C'] + decision_vars['VSTPE10'] + decision_vars['VSPUR1015LM'] == 356.7
# problem2 += decision_vars['VSHI4BTF'] <= 237.8
# #print(problem)
# problem2.solve()
# # print(p.value(problem2.objective))
# # for v in problem2.variables():
# #     print(v.name, "=", v.varValue)
    
# total_cost2 = p.value(problem2.objective) + sum([constants2[i]*costs[i] for i in constants2])
# total_mass2 = sum([v.varValue for v in problem2.variables()]) + sum(constants2.values())
# cost_per_unit2 = total_cost2/total_mass2
# # cost_per_unit2

# polymers2 = pd.Series([v.varValue for v in problem2.variables()], index = [v.name for v in problem1.variables()])
# polymers2 = pd.DataFrame(polymers2, columns=['LP2'])
# constraints2 = pd.DataFrame.from_dict(constants2, orient='index', columns=['LP2'])
# lp2frame = pd.concat([polymers2, constraints2], axis =0).T
# lp2frame['Batch Size (kg)'] = total_mass2
# lp2frame['Cost per kg (£)'] = cost_per_unit2
# lp2frame = lp2frame.rename(columns = {'decision_VPOLSHI17C':'VPOLSHI17C'}).rename(columns = {'decision_VSHI4BTF':'VSHI4BTF'}).rename(columns = {'decision_VSPUR1015LM':'VSPUR1015LM'}).rename(columns = {'decision_VSTPE10':'VSTPE10'})
# # lp2frame


# ### LP3 Polymer for constant VDAMO & VDINP ###

# constants3 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
#              'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VDAMO': 14.9, 'XDINP': 241.8}

# problem3 = p.LpProblem('dynamic_formulation_problem', p.LpMinimize)
# problem3 += p.lpSum([costs[i]*decision_vars[i] for i in decision_variables])
# problem3 += decision_vars['VSHI4BTF'] + decision_vars['VPOLSHI17C'] + decision_vars['VSTPE10'] + decision_vars['VSPUR1015LM'] == 356.7
# problem3 += decision_vars['VSHI4BTF'] <= 237.8
# #print(problem)
# problem3.solve()
# # print(p.value(problem3.objective))
# # for v in problem3.variables():
# #     print(v.name, "=", v.varValue)
    
# total_cost3 = p.value(problem3.objective) + sum([constants3[i]*costs[i] for i in constants3])
# total_mass3 = sum([v.varValue for v in problem3.variables()]) + sum(constants3.values())
# cost_per_unit3 = total_cost3/total_mass3
# # cost_per_unit3

# polymers3 = pd.Series([v.varValue for v in problem3.variables()], index = [v.name for v in problem1.variables()])
# polymers3 = pd.DataFrame(polymers3, columns=['LP3'])
# constraints3 = pd.DataFrame.from_dict(constants3, orient='index', columns=['LP3'])
# lp3frame = pd.concat([polymers3, constraints3], axis =0).T
# lp3frame['Batch Size (kg)'] = total_mass3
# lp3frame['Cost per kg (£)'] = cost_per_unit3
# lp3frame = lp3frame.rename(columns = {'decision_VPOLSHI17C':'VPOLSHI17C'}).rename(columns = {'decision_VSHI4BTF':'VSHI4BTF'}).rename(columns = {'decision_VSPUR1015LM':'VSPUR1015LM'}).rename(columns = {'decision_VSTPE10':'VSTPE10'})
# #lp3frame


# ### LP4 Polymer for constant VDAMO & VDINCH ###

# constants4 = {'VCRAYVALLACSLT': 19.5, 'VOMNYA5ML': 260, 'VHAKUENKACCRS': 780, 'VXL10': 65.0, 
#              'XTITAN2': 13.0, 'VCAT850': 3.3, 'VDIDP': 29.3, 'VDAMO': 14.9, 'VDINCH': 241.8}

# problem4 = p.LpProblem('dynamic_formulation_problem', p.LpMinimize)
# problem4 += p.lpSum([costs[i]*decision_vars[i] for i in decision_variables])
# problem4 += decision_vars['VSHI4BTF'] + decision_vars['VPOLSHI17C'] + decision_vars['VSTPE10'] + decision_vars['VSPUR1015LM'] == 356.7
# problem4 += decision_vars['VSHI4BTF'] <= 237.8
# #print(problem)
# problem4.solve()
# # print(p.value(problem4.objective))
# # for v in problem4.variables():
# #     print(v.name, "=", v.varValue)
    
# total_cost4 = p.value(problem4.objective) + sum([constants4[i]*costs[i] for i in constants4])
# total_mass4 = sum([v.varValue for v in problem4.variables()]) + sum(constants4.values())
# cost_per_unit4 = total_cost4/total_mass4

# polymers4 = pd.Series([v.varValue for v in problem4.variables()], index = [v.name for v in problem1.variables()])
# polymers4 = pd.DataFrame(polymers4, columns=['LP4'])
# constraints4 = pd.DataFrame.from_dict(constants4, orient='index', columns=['LP4'])
# lp4frame = pd.concat([polymers4, constraints4], axis =0).T
# lp4frame['Batch Size (kg)'] = total_mass4
# lp4frame['Cost per kg (£)'] = cost_per_unit4
# lp4frame = lp4frame.rename(columns = {'decision_VPOLSHI17C':'VPOLSHI17C'}).rename(columns = {'decision_VSHI4BTF':'VSHI4BTF'}).rename(columns = {'decision_VSPUR1015LM':'VSPUR1015LM'}).rename(columns = {'decision_VSTPE10':'VSTPE10'})

In [6]:
results = pd.Series([cost_per_unit1, cost_per_unit2, cost_per_unit3, cost_per_unit4], index =['LP1', 'LP2', 'LP3', 'LP4'])
pd.DataFrame(results).T

NameError: name 'cost_per_unit1' is not defined

In [17]:
lpframe = pd.concat([lp1frame, lp2frame, lp3frame, lp4frame, point_formulations], axis=0)
lpframe = lpframe[['VSTPE10', 'VPOLSHI17C', 'VSHI4BTF', 'VSPUR1015LM', 'VCRAYVALLACSLT',
       'VOMNYA5ML', 'VHAKUENKACCRS', 'VXL10', 'XTITAN2', 'VCAT850',
       'VDIDP', 'VAMMO', 'XDINP','VDINCH', 'VDAMO', 'Batch Size (kg)', 'Cost per kg (£)']]
lpframe = lpframe.fillna(0)
standard_cost = lpframe.iloc[4,16]

lpframe['Cost difference compared to standard per kg (£)'] = lpframe['Cost per kg (£)'] - standard_cost
lpframe['Cost difference compared to standard per kg (£)'] = lpframe['Cost difference compared to standard per kg (£)'].round(3)
lpframe = lpframe.sort_values('Cost per kg (£)')
lpframe

,VSTPE10,VPOLSHI17C,VSHI4BTF,VSPUR1015LM,VCRAYVALLACSLT,VOMNYA5ML,VHAKUENKACCRS,VXL10,XTITAN2,VCAT850,VDIDP,VAMMO,XDINP,VDINCH,VDAMO,Batch Size (kg),Cost per kg (£),Cost difference compared to standard per kg (£)
LP1,0.0,118.9,237.8,0.0,19.50,260.0,780.0,65.0,13.0,3.3,29.3,17.9,241.8,0.0,0.0,1786.50,1.251718,-0.013
LP3,0.0,118.9,237.8,0.0,19.50,260.0,780.0,65.0,13.0,3.3,29.3,0.0,241.8,0.0,14.9,1783.50,1.264270,-0.000
Standard,0.0,118.9,237.8,0.0,19.50,260.0,780.0,65.0,13.0,3.3,29.3,0.0,241.8,0.0,14.9,1783.50,1.264270,0.000
LP2,0.0,118.9,237.8,0.0,19.50,260.0,780.0,65.0,13.0,3.3,29.3,17.9,0.0,241.8,0.0,1786.50,1.375426,0.111
LP4,0.0,118.9,237.8,0.0,19.50,260.0,780.0,65.0,13.0,3.3,29.3,0.0,0.0,241.8,14.9,1783.50,1.388187,0.124
1,356.7,0.0,0.0,0.0,9.75,260.0,780.0,65.0,13.0,0.0,0.0,17.9,271.1,0.0,0.0,1773.45,1.482999,0.219
3,356.7,0.0,0.0,0.0,19.50,260.0,780.0,65.0,13.0,0.0,0.0,0.0,271.1,0.0,14.9,1780.20,1.528097,0.264
4,356.7,0.0,0.0,0.0,11.70,260.0,780.0,65.0,13.0,0.0,0.0,0.0,0.0,271.1,14.9,1772.40,1.642278,0.378
2,356.7,0.0,0.0,0.0,19.50,260.0,780.0,65.0,13.0,0.0,0.0,17.9,0.0,271.1,0.0,1783.20,1.654033,0.390
